<a href="https://colab.research.google.com/github/Bobisreallyme/TestRepo/blob/main/Cleaned_DB_updater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chat_downloader

In [ ]:
ODS_file_toupdate = 'Your_DB_file_toupdate.ods'

In [ ]:
import os
import sqlite3
from googleapiclient.discovery import build
from datetime import datetime
import numpy as np
import pandas as pd
from chat_downloader import ChatDownloader

In [ ]:
#EXTRACT THE URLS FROM THE DOCUMENT
def get_urls_database(db_document):
    Chatfile = sqlite3.connect(db_document)
    Cursorfile = Chatfile.cursor()
    Cursorfile.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_name = Cursorfile.fetchall()
    table_name = table_name[0][0]
    query = "SELECT URL FROM " + table_name
    all_urls = Cursorfile.execute(query)

    url_list = []
    for temp_urls in all_urls:
        temp_urls = temp_urls[0]
        url_list.append(temp_urls.split('&t=')[0])
    unique_urls = list(set(url_list))

    Cursorfile.close()

    return unique_urls,table_name,url_list

#GET CHAT FROM STREAM
def GetChatfromstream(urlinput):
    try:
        return ChatDownloader().get_chat(urlinput, message_groups=['messages', 'superchat'])
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

#CODE TO UPDATE DATABASE
def update_database(new_streams,db_document,table_name):
    Allchat = []

    import sqlite3
    conn = sqlite3.connect(db_document)
    batchsize = 100
    c = conn.cursor()

    id_values = []
    query_1 = "SELECT id FROM " + table_name
    c.execute(query_1)
    id_values_temp = c.fetchall()

    for i in range(0,len(id_values_temp)):
        id_values.append(id_values_temp[i][0])

    istart = max(id_values)+1

    i = istart
    j = 1


    query_2 = '''INSERT INTO {}
    (id, URL, TypeofMessages, ListofTimes, ListofBadges, ListofAuthors, ListofMessages, DonationAmount, DonationCurrency)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'''.format(table_name)


    for URL in new_streams:

        chat = GetChatfromstream(URL)

        batch = []
        print(j)
        if chat!=[]:
            for message in chat:
                try:
                    test = message['time_in_seconds']
                    if message['message_type'] == 'paid_message':
                        ListofMessages = message['message']
                        DonationAmount = message['money']['amount']
                        DonationCurrency = message['money']['currency']
                    elif message['message_type'] == 'paid_sticker':
                        ListofMessages = 'NULL'
                        DonationAmount = message['money']['amount']
                        DonationCurrency = message['money']['currency']
                    else:
                        ListofMessages = message['message']
                        DonationAmount = 'NULL'
                        DonationCurrency = 'NULL'

                    ListofAuthors = message['author']['name']

                    if 'badges' not in message['author']:
                        ListofBadges = 'NULL'
                    else:
                        ListofBadges = message['author']['badges'][0]['title']

                    ListofTimes = message['time_in_seconds']
                    TypeofMessages = message['message_type']

                except KeyError:

                    print('KeyError Encountered')
                    break

                batch.append((i,URL,TypeofMessages,ListofTimes,ListofBadges,ListofAuthors,ListofMessages,DonationAmount,DonationCurrency))
                i=i+1

                if len(batch) >= batchsize:
                    c.executemany(query_2, batch)
                    batch = []

            if batch:
                c.executemany(query_2, batch)
            j = j + 1
            conn.commit()

#GET THE FILES TO UPDATE
inputs_to_check = pd.read_excel(ODS_file_toupdate,header=None)
all_excel_file = inputs_to_check.iloc[:,1].tolist()
all_db_document = inputs_to_check.iloc[:,2].tolist()


#RUN UPDATING ON EACH FILE
for temp_index in range(len(all_excel_file)):
    db_document = all_db_document[temp_index]
    excel_file = all_excel_file[temp_index]
    Alldataframe = pd.read_excel(excel_file)
    ListofStreamUrls = Alldataframe.iloc[:,2].tolist()

    unique_urls,table_name,url_list = get_urls_database(db_document)

    last_database_index = [index for index,url_temp in enumerate(ListofStreamUrls) if url_temp in unique_urls]

    future_streams  = ListofStreamUrls[max(last_database_index)+1:len(ListofStreamUrls)]
    print(future_streams)

    new_streams = [url_temp for index_temp, url_temp in enumerate(future_streams) if url_temp not in unique_urls]
    print(new_streams)

    update_database(new_streams,db_document,table_name)